In [1]:
from model.model import ConvModel, SimCLR
import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from torchvision import transforms, datasets


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

In [3]:
composed_transforms = transforms.Compose([
      # transforms.RandomResizedCrop(size=28),
      # transforms.RandomHorizontalFlip(),
      # transforms.RandomGrayscale(p=0.2),
      transforms.ToTensor()])

In [4]:
train_dataset = datasets.MNIST('./datasets', train=True,
                               transform=composed_transforms,
                               download=True)
g = torch.Generator()
g.manual_seed(0)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./datasets\MNIST\raw\train-images-idx3-ubyte.gz to ./datasets\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./datasets\MNIST\raw\train-labels-idx1-ubyte.gz to ./datasets\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to ./datasets\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./datasets\MNIST\raw



In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=False,
    num_workers=12, pin_memory=True, drop_last=True, generator=g)

In [6]:
model = ConvModel(out_dim=10)
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

C:\Users\kacpe\Desktop\GB\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kacpe\Desktop\GB\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
with torch.cuda.device(gpu_index):
    simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler)
    simclr.train(train_loader)

KeyError: 'args'